In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
import yfinance as yf
from datetime import datetime, timedelta

In [ ]:
# Fetch historical stock price data for Netflix
ticker = "NFLX"
start_date = "2010-01-01"
end_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')  # Yesterday's date
netflix_data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Preprocess the data
training_data = netflix_data['Close'].values.reshape(-1, 1)
sc = MinMaxScaler(feature_range=(0, 1))
training_data_scaled = sc.fit_transform(training_data)

In [ ]:

# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, len(training_data_scaled)):
    X_train.append(training_data_scaled[i-60:i, 0])
    y_train.append(training_data_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:


# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=1))

In [ ]:
# Compile and fit the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
111/111 [==============================] - 16s 96ms/step - loss: 0.0077
Epoch 2/50
111/111 [==============================] - 10s 93ms/step - loss: 0.0026
Epoch 3/50
111/111 [==============================] - 9s 82ms/step - loss: 0.0024
Epoch 4/50
111/111 [==============================] - 10s 93ms/step - loss: 0.0020
Epoch 5/50
111/111 [==============================] - 10s 93ms/step - loss: 0.0018
Epoch 6/50
111/111 [==============================] - 10s 93ms/step - loss: 0.0020
Epoch 7/50
111/111 [==============================] - 10s 93ms/step - loss: 0.0018
Epoch 8/50
111/111 [==============================] - 9s 84ms/step - loss: 0.0018
Epoch 9/50
111/111 [==============================] - 11s 97ms/step - loss: 0.0016
Epoch 10/50
111/111 [==============================] - 11s 100ms/step - loss: 0.0014
Epoch 11/50
111/111 [==============================] - 10s 93ms/step - loss: 0.0014
Epoch 12/50
111/111 [==============================] - 10s 88ms/step - loss: 0.0015
Ep

In [ ]:



# Build the GRU model
gru_model = Sequential()
gru_model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
gru_model.add(Dropout(0.2))
gru_model.add(GRU(units=50, return_sequences=True))
gru_model.add(Dropout(0.2))
gru_model.add(GRU(units=50))
gru_model.add(Dropout(0.2))
gru_model.add(Dense(units=1))

In [ ]:

# Compile and fit the model
gru_model.compile(optimizer='adam', loss='mean_squared_error')
gru_model.fit(X_train, y_train, epochs=50, batch_size=32)


Epoch 1/50
111/111 [==============================] - 17s 98ms/step - loss: 0.0081
Epoch 2/50
111/111 [==============================] - 11s 99ms/step - loss: 0.0024
Epoch 3/50
111/111 [==============================] - 11s 98ms/step - loss: 0.0021
Epoch 4/50
111/111 [==============================] - 10s 93ms/step - loss: 0.0018
Epoch 5/50
111/111 [==============================] - 10s 89ms/step - loss: 0.0016
Epoch 6/50
111/111 [==============================] - 11s 98ms/step - loss: 0.0016
Epoch 7/50
111/111 [==============================] - 11s 97ms/step - loss: 0.0015
Epoch 8/50
111/111 [==============================] - 11s 98ms/step - loss: 0.0014
Epoch 9/50
111/111 [==============================] - 11s 98ms/step - loss: 0.0014
Epoch 10/50
111/111 [==============================] - 10s 87ms/step - loss: 0.0013
Epoch 11/50
111/111 [==============================] - 11s 96ms/step - loss: 0.0012
Epoch 12/50
111/111 [==============================] - 11s 98ms/step - loss: 0.0011
E

In [ ]:

# Predictions
inputs = training_data_scaled[-60:]  # Select the last 60 data points
X_test = []
X_test.append(inputs)  # Append the last 60 data points to X_test
X_test = np.array(X_test)

In [ ]:


lstm_predicted_stock_price = lstm_model.predict(X_test)
lstm_predicted_stock_price = sc.inverse_transform(lstm_predicted_stock_price)



1/1 [==============================] - 2s 2s/step


In [ ]:
lstm_predicted_stock_price

array([[595.67773]], dtype=float32)

In [ ]:

gru_predicted_stock_price = gru_model.predict(X_test)
gru_predicted_stock_price = sc.inverse_transform(gru_predicted_stock_price)

1/1 [==============================] - 0s 33ms/step


In [ ]:
gru_predicted_stock_price

array([[617.9961]], dtype=float32)

In [ ]:
# Reshape predicted prices to 1D array
lstm_predicted_stock_price = lstm_predicted_stock_price.flatten()
gru_predicted_stock_price = gru_predicted_stock_price.flatten()

In [ ]:
# Calculate Mean Squared Percentage Error (MSPE) for LSTM model
lstm_mspe = np.mean(((lstm_predicted_stock_price - netflix_data['Close'][-len(lstm_predicted_stock_price):]) / netflix_data['Close'][-len(lstm_predicted_stock_price):])**2)
print("MSPE for LSTM model:", lstm_mspe)

MSPE for LSTM model: 0.002771524538594821


In [ ]:
# Calculate Mean Squared Percentage Error (MSPE) for GRU model
gru_mspe = np.mean(((gru_predicted_stock_price - netflix_data['Close'][-len(gru_predicted_stock_price):]) / netflix_data['Close'][-len(gru_predicted_stock_price):])**2)
print("MSPE for GRU model:", gru_mspe)


MSPE for GRU model: 0.0002941420316762306


In [ ]:
# Calculate Mean Absolute Percentage Error (MAPE) for LSTM model
lstm_mape = np.mean(np.abs((lstm_predicted_stock_price - netflix_data['Close'][-len(lstm_predicted_stock_price):]) / netflix_data['Close'][-len(lstm_predicted_stock_price):])) * 100
print("MAPE for LSTM model:", lstm_mape)

MAPE for LSTM model: 5.26452708093977


In [ ]:
# Calculate Mean Absolute Percentage Error (MAPE) for GRU model
gru_mape = np.mean(np.abs((gru_predicted_stock_price - netflix_data['Close'][-len(gru_predicted_stock_price):]) / netflix_data['Close'][-len(gru_predicted_stock_price):])) * 100
print("MAPE for GRU model:", gru_mape)

MAPE for GRU model: 1.7150569427171525


In [ ]:
# Calculate RMSE for LSTM model
lstm_rmse = np.sqrt(np.mean((lstm_predicted_stock_price - netflix_data['Close'][-len(lstm_predicted_stock_price):])**2))
print("RMSE for LSTM model:", lstm_rmse)

RMSE for LSTM model: 33.102294921875


In [ ]:
gru_rmse = np.sqrt(np.mean((gru_predicted_stock_price - netflix_data['Close'][-len(gru_predicted_stock_price):])**2))
print("RMSE for GRU model:", gru_rmse)

RMSE for GRU model: 10.783935546875
